# GIS 5571 Lab 1

## Kyle Smith, smi02542@umn.edu

----

#### Python packages

In [1]:
# !pip install arcgis
# !pip install shapely
# !pip install ipywidgets 

In [2]:
import os
import json
import requests
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from osgeo import ogr
from pyproj import Transformer
from shapely import wkt
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureSet
from arcgis.geometry import Geometry, Polyline, SpatialReference
from arcgis.mapping import WebMap
from arcgis.widgets import MapView
from arcgis.geometry import Point
import shapefile
import ipywidgets  as widgets
import warnings

gis = GIS()

warnings.simplefilter(action='ignore', category=FutureWarning)

# * Download three datasets (one from each API), convert to spatially enabled databases & same coordinate reference system

#### 1. Minnesota Geospatial Commons API (Strategic_Highways)

Dataset: National Highway System, Truck Network, and Strategic Highway Network - https://gisdata.mn.gov/dataset/trans-federal-routes
Specifically, I will use the "Strategic_Highway_Network_in_Minnesota" feature service, which only includes roads identified as priority routes for national defense purposes. 

In [25]:
# Use CKAN to search MN Geospatial Commons for "trans-federal-routes" 
api = "https://gisdata.mn.gov/api/3/action/package_search?q=trans-federal-routes"

# Request to GET from the API
response = requests.get(api)

# Load the JSON extracted from API search results 
json_data = response.json()

# Find the path in JSON for 'trans-federal-routes'
zip_url = json_data['result']['results'][0]['resources'][0]['url']
zip_url

# Download the zip file
zip_file_path = 'trans_federal_routes.zip'
with requests.get(zip_url, stream=True) as r:
    with open(zip_file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
            
# Extract the ZIP file
extraction_path = "trans_federal_routes_data"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# List the extracted files
extracted_files = os.listdir(extraction_path)
#extracted_files

# looking for .... trans_federal_routes_data/Strategic_Highway_Network_in_Minnesota.shp
file_path = 'trans_federal_routes_data/Strategic_Highway_Network_in_Minnesota.shp'

# Make a Spatially Enabled DataFrame
sdf_Strategic_Highways = pd.DataFrame.spatial.from_featureclass(file_path)
#sdf_Strategic_Highways.head()

#check geometry and sr
sdf_Strategic_Highways.spatial.sr = {'wkid': 26915}
sdf_Strategic_Highways.spatial.set_geometry('SHAPE')
print(sdf_Strategic_Highways.spatial.sr)  
print(sdf_Strategic_Highways.spatial.geometry_type)  
print(sdf_Strategic_Highways.spatial.validate())
#sdf_Strategic_Highways.head()

{'wkid': 26915}
['polyline']
True


In [4]:
map1 = gis.map("Minnesota")

# Convert to FeatureCollection
feature_highways = sdf_Strategic_Highways.spatial.to_feature_collection()

# Add the Hhighways Feature to the map
map1.add_layer(feature_highways)

#map1

In [5]:
#make a csv
sdf_Strategic_Highways.to_csv('Lab_1__MNGeo_Strategic_Highways.csv', index=False)

#### 2. REST API (mn_counties)

Dataset: County boundaries in Minnesota (polygons) - https://webgis.dot.state.mn.us/65agsf1/rest/services/sdw_govnt/COUNTY/FeatureServer/0

In [26]:
# REST API URL
api = 'https://webgis.dot.state.mn.us/65agsf1/rest/services/sdw_govnt/COUNTY/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=4326&havingClause=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&returnTrueCurves=false&returnExceededLimitFeatures=false&quantizationParameters=&returnCentroid=false&timeReferenceUnknownClient=false&sqlFormat=none&resultType=&featureEncoding=esriDefault&datumTransformation=&f=pjson'

# Request data from the API
r = requests.get(api)
data = r.json()

# Create a FeatureSet from the JSON data
fs = FeatureSet.from_dict(data)
#data

# Create a Spatially Enabled DataFrame from the FeatureSet
sdf_mn_counties = fs.sdf


#check geometry and sr
sdf_mn_counties.spatial.set_geometry('SHAPE')
print(sdf_mn_counties.spatial.sr)  
print(sdf_mn_counties.spatial.geometry_type)  
print(sdf_mn_counties.spatial.validate())
#sdf_mn_counties.head()


{'wkid': 4326, 'latestWkid': 4326}
['polygon']
True


In [7]:
#make a csv
sdf_mn_counties.to_csv('Lab_1__REST_MN_counties.csv', index=False)

#### 3. NDAWN

Using available data from NDAWN, find the monthly average max temp and annual monthly min temp for a 48 month period at a NDAWN site in north Minnesota and south Minnesota

    -- NDAWN Station: Humboldt, MN (Station ID 4) - site in north Minnesota 
    -- NDAWN Station: Becker, MN (Station ID 118) - site in south Minnesota 

In [27]:
#Call api from NDAWN
api = 'https://ndawn.ndsu.nodak.edu/table.csv?station=118&station=4&variable=mdmxt&variable=mdmnt&year=2024&ttype=monthly&quick_pick=4_y&begin_date=2019-01&count=12'
#api is comma delimited csv file

#view api path on web (JSON) as it does not open as a data frame in current form


#columns / rows appear to be missalligned because raw NDAWNS csv header rows are merged
#To fix, skip rows 0, 1, 2, 4
#Column headers are row 3

#Pandas create a data frame and skip rows  0, 1, 2, 4
df = pd.read_csv(api, skiprows=[0, 1, 2, 4])

#For simplicity, Can also skip/drop columns 3,7,8,10,11 
df = df.drop(df.columns[[3,7,8,10,11]], axis=1)

NDAWN_station_data = pd.DataFrame(df)
#NDAWN_station_data

#convert latitude and longitude to point geometry
NDAWN_station_data = GeoAccessor.from_xy(NDAWN_station_data, x_column='Longitude', y_column='Latitude', sr=4326)

NDAWN_station_data.spatial.set_geometry('SHAPE')
print(NDAWN_station_data.spatial.sr)  
print(NDAWN_station_data.spatial.geometry_type)  
print(NDAWN_station_data.spatial.validate())


#View NDAWN_station_data
#Should be 96 rows and columns: Name / Latitude / Longitude / year / Month / Max / Min 
#NDAWN_station_data.head()

{'wkid': 4326}
['point']
True


In [9]:
# Now, calculate the mean of Max Temp and Min Temp for each station, and round to two decimal points so cleaner
# Group by 'Station Name' and calculate the mean for 'Avg Max Temp' and 'Avg Min Temp', rounding to 2 decimal places
sdf_grouped = df.groupby('Station Name')[['Avg Max Temp', 'Avg Min Temp', 'Latitude', 'Longitude']].mean().round(5)

#sdf_grouped

#convert latitude and longitude to point geometry
sdf_NDAWN = GeoAccessor.from_xy(sdf_grouped, x_column='Longitude', y_column='Latitude', sr=4326)
sdf_NDAWN

#check geometry and sr
#sdf_NDAWN.spatial.sr = {'wkid': 4326}

sdf_NDAWN.spatial.set_geometry('SHAPE')
print(sdf_NDAWN.spatial.sr)  
print(sdf_NDAWN.spatial.geometry_type)  
print(sdf_NDAWN.spatial.validate())

sdf_NDAWN

{'wkid': 4326}
['point']
True


,Avg Max Temp,Avg Min Temp,Latitude,Longitude,SHAPE
Station Name,,,,,
Becker,55.49881,35.49517,45.34399,-93.85014,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
Humboldt,50.26750,28.63329,48.88351,-97.15029,"{""spatialReference"": {""wkid"": 4326}, ""x"": -97...."


So the average max temperature for the 48 month period at Humboldt, MN (north Minnesota) is about 5.23 degrees cooler then the the average max temperature for Becker, MN (south Minnesota). 
Further, the average min termperature for the 48 month period at Becker, MN (south Minnesota) is about 6.87 degrees warmer then the the average min temperature for Humboldt, MN (north Minnesota).


In [10]:
#Fully monthly Station data for the two sites can be downloaded to a csv
NDAWN_station_data.to_csv('Lab_1_NDAWN_station_data.csv', index=False)

#Station averages can be downloaded to a csv
sdf_NDAWN.to_csv('Lab_1__NDAWNS_merged.csv', index=False)

# * Spatially join two of the three datasets & print head of the table showing the merged attributes

I want to spatially join Minnesota Counties and NDAWN - Becker & Humboldt, MN datasets. From the joined database, I can identify the county names for each of the NDAWN sites used here. 

      sdf_mn_counties 
        -polygon geometry in column = 'SHAPE'{"rings}
        -county name in column = 'COUNTY_NAME'

      NDAWN_station_data
        -station name in column = 'Station Name'
        -point geometry in column = 'SHAPE'
        
      Find NDAWN points that are within sdf_mn_counties polygons


#### Simple map

In [ ]:
# Create the map object centered on Minnesota
map4 = gis.map("Minnesota")

# Convert NDAWN data to FeatureCollection with renderer and add to map
feature_NDAWN = sdf_NDAWN.spatial.to_feature_collection()
map4.add_layer(feature_NDAWN)

# Counties: Convert the spatial dataframes from above to a FeatureCollection and add to map
feature_counties = sdf_mn_counties.spatial.to_feature_collection()
map4.add_layer(feature_counties)

#map is at the very bottom

#### Spatial Join

In [ ]:
print("sdf_mn_counties: ")

#Check spatial reference of sdf_mn_counties 
print(sdf_mn_counties.spatial.sr)  

#need to reproject sdf_mn_counties from 26915 to 4326
#sdf_mn_counties.spatial.sr = {'wkid': 4326}
sdf_mn_counties.spatial.project(SpatialReference(4326))

#Check reprojected spatial reference of sdf_mn_counties 
print(sdf_mn_counties.spatial.sr)  
print(sdf_mn_counties.spatial.geometry_type)  
print(sdf_mn_counties.spatial.validate())
print(sdf_mn_counties.columns)
#sdf_mn_counties.head()

In [ ]:


#Need to look at NDAWN_station_data and eliminate duplicate rows before spatial join
NDAWN_station_data = NDAWN_station_data.drop_duplicates(subset='Station Name', keep='first')
# reset indices 
NDAWN_station_data = NDAWN_station_data.reset_index(drop=True) 
#NDAWN_station_data.head()

NDAWN_station_data.spatial.project(SpatialReference(26915))

#Check spatial reference of NDAWN_station_data
print("NDAWN_station_data: ")
#NDAWN_station_data.spatial.sr 
NDAWN_station_data.spatial.geometry_type
NDAWN_station_data.spatial.validate()
#NDAWN_station_data.columns
#NDAWN_station_data.head()



In [ ]:
#Reproject and confirm 
sdf_NDAWN.spatial.project(SpatialReference(4326))
sdf_mn_counties.spatial.project(SpatialReference(4326))

print(sdf_NDAWN.spatial.sr)
print(sdf_mn_counties.spatial.sr)  


In [ ]:
# reset indices 
sdf_mn_counties = sdf_mn_counties.reset_index(drop=True)


# Perform the spatial join using 'intersects'
spatial_join = sdf_mn_counties.spatial.join(NDAWN_station_data, how='inner', op='intersects')



# Check the results of the spatial join
print("Minnesota County names for NDAWN stations:")

print(spatial_join[['Station Name', 'COUNTY_NAME']])


In [16]:
#spatial_join.head()
#sdf_NDAWN.columns
#sdf_mn_counties.columns

To summarize...

The two NDAWN sites in Minnesota choosen for this project are:
    
        -- Becker, MN is in Sherburne County, Minnesota
        -- Humboldt, MN is in Kittson County, Minnesota

#### Merged table head

In [23]:
spatial_join.head()

,OBJECTID,COUNTY_NAME,COUNTY_CODE,COUNTY_FIPS55_CODE,COUNTY_GNIS_FEATURE_ID,ATP_CODE,Shape__Area,Shape__Length,SHAPE,index_right,Station Name,Latitude,Longitude,Year,Month,Avg Max Temp,Avg Min Temp
0,253,Kittson,35,069,659480,2,0.0,0.0,"{""rings"": [[[-97.0839237552649, 49.00033853807...",1,Humboldt,48.88351,-97.15029,2020,10,45.515,25.838
1,178,Sherburne,71,141,659515,3,0.0,0.0,"{""rings"": [[[-94.03581736621345, 45.5590780377...",0,Becker,45.34399,-93.85014,2020,10,49.622,32.215


In [18]:
#Saved spatial joined table
spatial_join.to_csv('Lab_1__Joined_Set.csv', index=False)

# * Save the integrated dataset to a geodatabase

In [19]:
# Save the Statial Join DataFrame to geodatabase
#spatial_join

Lab_1__Joined_Set = spatial_join
feature_class_name = 'Lab_1__Joined_Set'  
output_path_join = spatial_join.spatial.to_featureclass(location=f'Lab_1__Joined_Set.gdb/{feature_class_name}')
output_path_join

'/arcgis/Lab_1__Joined_Set.gdb/Lab_1__Joined_Set.shp'

In [24]:
# Display the map
map4

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)